https://docs.sqlalchemy.org/en/latest/core/tutorial.html

In [1]:
import sqlalchemy

print("バージョンチェック")
print(sqlalchemy.__version__)


バージョンチェック
1.2.12


In [2]:
print("接続")
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)


接続


In [3]:
print("テーブル作成と定義")
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
metadata = MetaData()
# ユーザーテーブル
users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('fullname', String),
)
# アドレステーブル
addresses = Table('addresses', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', None, ForeignKey('users.id')),
    Column('email_address', String, nullable=False)
)
# 作成
metadata.create_all(engine)

テーブル作成と定義
2018-12-16 05:02:05,432 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-12-16 05:02:05,433 INFO sqlalchemy.engine.base.Engine ()
2018-12-16 05:02:05,435 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-12-16 05:02:05,437 INFO sqlalchemy.engine.base.Engine ()
2018-12-16 05:02:05,440 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-12-16 05:02:05,442 INFO sqlalchemy.engine.base.Engine ()
2018-12-16 05:02:05,444 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-12-16 05:02:05,445 INFO sqlalchemy.engine.base.Engine ()
2018-12-16 05:02:05,449 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2018-12-16 05:02:05,450 INFO sqlalchemy.engine.base.Engine ()
2018-12-16 05:02:05,452 INFO sqlalchemy.engine.base.Engine COMMIT
2018-12-16 05:02:05,459 INFO sqlalchem

# insert

In [8]:
ins = users.insert()
print(str(ins))

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [9]:
ins.compile().params

{'fullname': None, 'id': None, 'name': None}

In [10]:
ins = users.insert().values(name='jack', fullname='Jack Jones')
print(str(ins))

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


In [11]:
ins.compile().params

{'fullname': 'Jack Jones', 'name': 'jack'}

# 実行

In [12]:
conn = engine.connect()
print(conn)

In [13]:
result = conn.execute(ins)
print(result)

2018-12-16 05:03:35,716 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-12-16 05:03:35,718 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Jones')
2018-12-16 05:03:35,720 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
ins.bind = engine
print(str(ins))

INSERT INTO users (name, fullname) VALUES (?, ?)


In [15]:
result.inserted_primary_key

[1]

# 複数行insert

In [16]:
ins = users.insert()
conn.execute(ins, id=2, name='wendy', fullname='Wendy Williams')

2018-12-16 05:05:55,131 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2018-12-16 05:05:55,134 INFO sqlalchemy.engine.base.Engine (2, 'wendy', 'Wendy Williams')
2018-12-16 05:05:55,136 INFO sqlalchemy.engine.base.Engine COMMIT


In [19]:
from sqlalchemy.sql import select
s = select([users])
result = conn.execute(s)
for row in result:
    print(row)

2018-12-16 05:07:14,576 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-12-16 05:07:14,578 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones')
(2, 'wendy', 'Wendy Williams')


In [20]:
conn.execute(addresses.insert(), [
   {'user_id': 1, 'email_address' : 'jack@yahoo.com'},
   {'user_id': 1, 'email_address' : 'jack@msn.com'},
   {'user_id': 2, 'email_address' : 'www@www.org'},
   {'user_id': 2, 'email_address' : 'wendy@aol.com'},
])

2018-12-16 05:08:27,799 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2018-12-16 05:08:27,801 INFO sqlalchemy.engine.base.Engine ((1, 'jack@yahoo.com'), (1, 'jack@msn.com'), (2, 'www@www.org'), (2, 'wendy@aol.com'))
2018-12-16 05:08:27,804 INFO sqlalchemy.engine.base.Engine COMMIT


In [22]:
from sqlalchemy.sql import select
s = select([addresses])
result = conn.execute(s)
for row in result:
    print(row)

2018-12-16 05:08:56,383 INFO sqlalchemy.engine.base.Engine SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses
2018-12-16 05:08:56,384 INFO sqlalchemy.engine.base.Engine ()
(1, 1, 'jack@yahoo.com')
(2, 1, 'jack@msn.com')
(3, 2, 'www@www.org')
(4, 2, 'wendy@aol.com')


# select

In [23]:
from sqlalchemy.sql import select
s = select([users])
result = conn.execute(s)
for row in result:
    print(row)

2018-12-16 05:10:05,151 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-12-16 05:10:05,152 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones')
(2, 'wendy', 'Wendy Williams')


In [24]:
result = conn.execute(s)
#１個取る
row = result.fetchone()
print("name:", row['name'], "; fullname:", row['fullname'])

2018-12-16 05:10:25,052 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-12-16 05:10:25,053 INFO sqlalchemy.engine.base.Engine ()
name: jack ; fullname: Jack Jones


In [25]:
#rowはインデックスでも取得できるよ
print("name:", row[1], "; fullname:", row[2])

name: jack ; fullname: Jack Jones


In [26]:
for row in conn.execute(s):
    print("name:", row[users.c.name], "; fullname:", row[users.c.fullname])

2018-12-16 05:12:43,640 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-12-16 05:12:43,642 INFO sqlalchemy.engine.base.Engine ()
name: jack ; fullname: Jack Jones
name: wendy ; fullname: Wendy Williams


In [27]:
#破棄
result.close()

In [29]:
s = select([users.c.name, users.c.fullname])
result = conn.execute(s)
for row in result:
    print(row)
result.close()

2018-12-16 05:15:19,722 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2018-12-16 05:15:19,723 INFO sqlalchemy.engine.base.Engine ()
('jack', 'Jack Jones')
('wendy', 'Wendy Williams')


In [30]:
#これの方が短い書き方
for row in conn.execute(select([users, addresses])):
    print(row)

2018-12-16 05:15:59,177 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses
2018-12-16 05:15:59,178 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(1, 'jack', 'Jack Jones', 3, 2, 'www@www.org')
(1, 'jack', 'Jack Jones', 4, 2, 'wendy@aol.com')
(2, 'wendy', 'Wendy Williams', 1, 1, 'jack@yahoo.com')
(2, 'wendy', 'Wendy Williams', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


In [31]:
 s = select([users, addresses]).where(users.c.id == addresses.c.user_id)
for row in conn.execute(s):
    print(row)

2018-12-16 05:16:30,536 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2018-12-16 05:16:30,537 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


In [32]:
#where句の==の中身を見るにはstrに変換する
users.c.id == addresses.c.user_id

In [33]:
str(users.c.id == addresses.c.user_id)

'users.id = addresses.user_id'

# 演算子

In [34]:
print(users.c.id == addresses.c.user_id)

users.id = addresses.user_id


In [35]:
#7のidデータを取得
print(users.c.id == 7)

users.id = :id_1


In [36]:
(users.c.id == 7).compile().params

{'id_1': 7}

In [37]:
print(users.c.id != 7)

users.id != :id_1


In [38]:
print(users.c.name == None)

users.name IS NULL


In [39]:
print('fred' > users.c.name)

users.name < :name_1


In [40]:
print(users.c.id + addresses.c.id)

users.id + addresses.id


In [41]:
print('fred' > users.c.name)

users.name < :name_1


In [42]:
print(users.c.id + addresses.c.id)

users.id + addresses.id


In [43]:
print((users.c.name + users.c.fullname).compile(bind=create_engine('mysql://'))) 

concat(users.name, users.fullname)


In [50]:
#id==1で絞る
s = select([users, addresses]).where(users.c.id == 1)
for row in conn.execute(s):
    print(row)

2018-12-16 05:31:43,280 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = ?
2018-12-16 05:31:43,282 INFO sqlalchemy.engine.base.Engine (1,)
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(1, 'jack', 'Jack Jones', 3, 2, 'www@www.org')
(1, 'jack', 'Jack Jones', 4, 2, 'wendy@aol.com')


Wehre句にand


In [64]:
from sqlalchemy import select, and_
#email_addressとidで絞る
s = select([users, addresses]).\
    where(and_(users.c.id == 1, addresses.c.email_address == "jack@yahoo.com"))

for row in conn.execute(s):
    print(row)

2018-12-16 05:36:36,458 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = ? AND addresses.email_address = ?
2018-12-16 05:36:36,459 INFO sqlalchemy.engine.base.Engine (1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')


Wehre句にor


In [65]:
from sqlalchemy.sql import and_, or_, not_
from sqlalchemy import select, and_
#email_addressとidで絞る
s = select([users, addresses]).\
    where(or_(users.c.id == 1, addresses.c.email_address == "jack@yahoo.com"))

for row in conn.execute(s):
    print(row)

2018-12-16 05:37:50,697 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = ? OR addresses.email_address = ?
2018-12-16 05:37:50,699 INFO sqlalchemy.engine.base.Engine (1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(1, 'jack', 'Jack Jones', 3, 2, 'www@www.org')
(1, 'jack', 'Jack Jones', 4, 2, 'wendy@aol.com')
(2, 'wendy', 'Wendy Williams', 1, 1, 'jack@yahoo.com')


# 結合

In [66]:
from sqlalchemy.sql import and_, or_, not_
print(and_(
        users.c.name.like('j%'),
        users.c.id == addresses.c.user_id,
        or_(
             addresses.c.email_address == 'wendy@aol.com',
             addresses.c.email_address == 'jack@yahoo.com'
        ),
        not_(users.c.id > 5)
      )
 )

users.name LIKE :name_1 AND users.id = addresses.user_id AND (addresses.email_address = :email_address_1 OR addresses.email_address = :email_address_2) AND users.id <= :id_1


In [67]:
s = select([(users.c.fullname +
              ", " + addresses.c.email_address).
               label('title')]).\
       where(
          and_(
              users.c.id == addresses.c.user_id,
              users.c.name.between('m', 'z'),
              or_(
                 addresses.c.email_address.like('%@aol.com'),
                 addresses.c.email_address.like('%@msn.com')
              )
          )
       )
conn.execute(s).fetchall()

2018-12-16 05:43:51,356 INFO sqlalchemy.engine.base.Engine SELECT users.fullname || ? || addresses.email_address AS title 
FROM users, addresses 
WHERE users.id = addresses.user_id AND users.name BETWEEN ? AND ? AND (addresses.email_address LIKE ? OR addresses.email_address LIKE ?)
2018-12-16 05:43:51,357 INFO sqlalchemy.engine.base.Engine (', ', 'm', 'z', '%@aol.com', '%@msn.com')


[('Wendy Williams, wendy@aol.com',)]